In [1]:
library(dplyr)
library(Seurat)
library(patchwork)
library(repr)
library(hdf5r)
library(MAST)
library(DoubletFinder)
library(tidyverse)
library(ggplot2)
library(ggrepel)
library(SingleR)
library(celldex)
library(clustree)
library(VennDiagram)
library(edgeR)
library(WGCNA)
library(hdWGCNA)
library(proxy)

Warning message:
"package 'dplyr' was built under R version 4.2.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching SeuratObject

Warning message:
"package 'patchwork' was built under R version 4.2.3"
Warning message:
"package 'repr' was built under R version 4.2.2"
Warning message:
"package 'hdf5r' was built under R version 4.2.3"
Loading required package: SingleCellExperiment

Warning message:
"package 'SingleCellExperiment' was built under R version 4.2.2"
Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats

Warning message:
"package 'matrixStats' was built under R version 4.2.3"

Attaching package: 'matrixStats'


The following object is masked from 'package:dplyr':

    count



Attaching package: 'MatrixGenerics'


The following objects are maske




Attaching package: 'WGCNA'


The following object is masked from 'package:IRanges':

    cor


The following object is masked from 'package:S4Vectors':

    cor


The following object is masked from 'package:stats':

    cor


Warning message:
"package 'proxy' was built under R version 4.2.2"

Attaching package: 'proxy'


The following object is masked from 'package:IRanges':

    as.matrix


The following object is masked from 'package:S4Vectors':

    as.matrix


The following objects are masked from 'package:stats':

    as.dist, dist


The following object is masked from 'package:base':

    as.matrix




# 空转部分差异基因绘图

In [2]:
section_ids <- c('ST_1M_AD_1_1', 'ST_1M_AD_1_2', 'ST_1M_WT_1_1', 'ST_1M_WT_1_2', 
                   'ST_3M_AD_1_1', 'ST_3M_AD_1_2', 'ST_3M_WT_1_1', 'ST_3M_WT_1_2',
                   'ST_6M_AD_2_1', 'ST_6M_AD_2_2', 'ST_6M_WT_2_1', 'ST_6M_WT_2_2',
                   'ST_15M_AD_2_1', 'ST_15M_AD_2_2', 'ST_15M_WT_2_1', 'ST_15M_WT_2_2')

# 读取16张切片的最终聚类结果

In [3]:
# 读取数据
Batch_list <- list()
i <- 1
for (section_id in section_ids){
    # The path to Data
    input_dir0 <- paste0('../AD_ST_Multiome/st/', section_id)
    #  读取计数矩阵
    adata <- Seurat::Load10X_Spatial(input_dir0)
    adata@meta.data$orig.ident <- section_id
    
    # 读取STAligner的聚类结果 ， 并根据 batch_name 分配给不同的 section
    final_results = read.csv(
        paste0('./results/最终聚类结果/',section_ids[i],'.csv'),
        row.names=1,
        sep=',',
        header=1)
    
    # 放入adata的结果里面
    adata@meta.data$STAligner <- final_results[rownames(adata@meta.data), 'hand_lebel']
    adata@meta.data$STAligner <- as.character(adata@meta.data$STAligner)
       
    names(adata@images) <- section_id
    Batch_list[[i]] <- adata
    i <- i + 1
}

In [4]:
# 将数据merge到一起
datas <- merge(
    x <- Batch_list[[1]], # [[1]]表示取列表的第一个元素
    y <- Batch_list[-1],  # -1 表示除了第一个元素以外剩下的列表
)

Warning message in CheckDuplicateCellNames(object.list = objects):
"Some cell names are duplicated across objects provided. Renaming to enforce unique cell names."


In [5]:
# 疾病信息
datas@meta.data$if_disease <- sapply(datas@meta.data$orig.ident, function(x){
    x <- strsplit(x, '_')[[1]][3]
})

# 月份信息
datas@meta.data$month <- sapply(datas@meta.data$orig.ident, function(x){
    x <- strsplit(x, '_')[[1]][2]
})

In [6]:
# 过略掉没有聚类结果的点
datas@meta.data$if_choose <- 1
datas@meta.data[is.na(datas@meta.data$STAligner), 'if_choose'] <- 0
datas@meta.data

,orig.ident,nCount_Spatial,nFeature_Spatial,STAligner,if_disease,month,if_choose
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<dbl>
AAACACCAATAACTGC-1_1,ST_1M_AD_1_1,44392,7554,L5,AD,1M,1
AAACAGAGCGACTCCT-1_1,ST_1M_AD_1_1,43837,8155,PIR,AD,1M,1
AAACCGGGTAGGTACC-1_1,ST_1M_AD_1_1,38131,8183,CA-2,AD,1M,1
AAACCGTTCGTCCAGG-1_1,ST_1M_AD_1_1,30094,6690,CC,AD,1M,1
AAACCTCATGAAGTTG-1_1,ST_1M_AD_1_1,36160,7413,TH-1,AD,1M,1
AAACGAAGAACATACC-1_1,ST_1M_AD_1_1,54396,8681,sAMY,AD,1M,1
AAACGAGACGGTTGAT-1_1,ST_1M_AD_1_1,35128,6857,STRD-2,AD,1M,1
AAACTGCTGGCTCCAA-1_1,ST_1M_AD_1_1,30042,6463,CC,AD,1M,1
AAACTTGCAAACGTAT-1_1,ST_1M_AD_1_1,63338,8692,CA-1,AD,1M,1


In [7]:
# 过滤掉不在组织切片上的点
datas <- subset(datas, if_choose==1)
datas

An object of class Seurat 
32285 features across 42956 samples within 1 assay 
Active assay: Spatial (32285 features, 0 variable features)
 16 images present: ST_1M_AD_1_1, ST_1M_AD_1_2, ST_1M_WT_1_1, ST_1M_WT_1_2, ST_3M_AD_1_1, ST_3M_AD_1_2, ST_3M_WT_1_1, ST_3M_WT_1_2, ST_6M_AD_2_1, ST_6M_AD_2_2, ST_6M_WT_2_1, ST_6M_WT_2_2, ST_15M_AD_2_1, ST_15M_AD_2_2, ST_15M_WT_2_1, ST_15M_WT_2_2

# 每个月份都计算一遍（重要！！！）

In [8]:
month_type <- '1M'

In [9]:
datas <- subset(datas, month==month_type)
# datas@meta.data

# 合并区域

In [10]:
# STAligner 空间域识别结果
STAligner_results <- c('L2/3','CTXsp-1','DG','CA-2','sAMY','Fxs+Int','HY','PIR','L5',
                       'CC','SI','TH-3','CA-1','L6','L1','VL','DORsm','TH-2','CTXsp-2',
                       'TH-1','opt','STRD-1','L4','STRD-2','RT')

In [11]:
# 合并后的分区的名称
re_region_name <- c('Cerebral cortex','Cerebral nuclei','Hippocampal region','Hippocampal region','Cerebral nuclei','fiber tracts','Hypothalamus','Cerebral cortex','Cerebral cortex',
                    'fiber tracts','Cerebral nuclei','Thalamus','Hippocampal region','Cerebral cortex','Cerebral cortex','VL','Thalamus','Thalamus','Cerebral nuclei',
                    'Thalamus','fiber tracts','Cerebral nuclei','Cerebral cortex','Cerebral nuclei','Thalamus')

In [12]:
datas@meta.data$STAligner <- plyr::mapvalues(datas@meta.data$STAligner, from = STAligner_results, to = re_region_name)

# 按照是否患病分组

In [13]:
# 样本分组
case_clusters = list()
control_clusters = list()
datas_case = subset(datas, if_disease=='AD')
datas_control = subset(datas, if_disease=='WT')

for (i in 1:length(unique(datas@meta.data$STAligner))) {
    case_clusters[[i]] = subset(datas_case, STAligner==unique(datas@meta.data$STAligner)[i])
    control_clusters[[i]] = subset(datas_control, STAligner==unique(datas@meta.data$STAligner)[i])
}
names(case_clusters) <- unique(datas@meta.data$STAligner)
names(control_clusters) <- unique(datas@meta.data$STAligner)

In [14]:
# 提取表达矩阵存放于列表之中
data_count_case = list()
data_count_control = list()

# 提取计数矩阵, 并转化为dataframe 以便后续处理操作
#  data @assays$RNA@counts存的是表达矩阵，  rownames (data @assays$RNA@meta.features)存的是相应的基因名
for (i in 1:length(unique(datas@meta.data$STAligner))) {
    data_count_case[[i]] = data.frame(case_clusters[[i]]@assays$Spatial@counts, row.names = rownames(case_clusters[[i]]@assays$Spatial@meta.features))
    data_count_control[[i]] = data.frame(control_clusters[[i]]@assays$Spatial@counts, row.names = rownames(control_clusters[[i]]@assays$Spatial@meta.features))
}

In [15]:
# # 记录基因表达量和的列表
# gene_sum_case = list()
# gene_sum_control = list()

# # 按行求和： 也就是求每个基因的总表达量之和，然后重新创建一个dataframe，行名是基因，列名是样本， 也就是将矩阵压缩为一列
# for (i in 1:length(unique(datas@meta.data$STAligner))) {
#     gene_sum_case[[i]] = data.frame(rowSums(data_count_case[[i]]), row.names = rownames(data_count_case[[i]]))
#     colnames(gene_sum_case[[i]]) = paste0('case_cluster_',i)
#     gene_sum_control[[i]] = data.frame(rowSums(data_count_control[[i]]), row.names = rownames(data_count_control[[i]]))
#     colnames(gene_sum_control[[i]]) = paste0('control_cluster_',i)
# }
# 记录基因表达量和的列表
gene_sum_case = list()
gene_sum_control = list()

# 按行求和： 也就是求每个基因的总表达量之和，然后重新创建一个dataframe，行名是基因，列名是样本， 也就是将矩阵压缩为一列
for (i in 1:length(unique(datas@meta.data$STAligner))) {
    gene_sum_case[[i]] = data.frame(rowSums(data_count_case[[i]]), row.names = rownames(data_count_case[[i]]))
#     colnames(gene_sum_case[[i]]) = paste0('AD_cluster_',i)
    colnames(gene_sum_case[[i]]) = paste0('AD')
    gene_sum_control[[i]] = data.frame(rowSums(data_count_control[[i]]), row.names = rownames(data_count_control[[i]]))
#     colnames(gene_sum_control[[i]]) = paste0('WT_cluster_',i)
    colnames(gene_sum_control[[i]]) = paste0('WT')
}

In [16]:
# 记录差异基因的列表
DE_analyze_up = list()
DE_analyze_down = list()

# group = c('control', 'case')
group <- c('WT', 'AD')

for (i in 1:length(unique(datas@meta.data$STAligner))) {
    # 将要差异表达的两个矩阵merge到一起， 按照原先的行名（也就是基因名）
    count_after_merge = merge(gene_sum_case[[i]], gene_sum_control[[i]], by = 'row.names', all=TRUE)
    # 由于merge完以后会添加一列（row.names），因此需要重新取出第一列作为行名
    rownames(count_after_merge)=count_after_merge[,1]  
    # 删除第一列
    count_after_merge=count_after_merge[,-1]    
    # 将NA值改为0
    count_after_merge[is.na(count_after_merge)] = 0
    
    # 构建含有组别标记的DGEList
    y <- edgeR::DGEList(counts=count_after_merge, group=group)

    # 自动过滤低表达
    keep.exprs<- edgeR::filterByExpr(y, group=group)
    y <- y[keep.exprs,,keep.lib.sizes=FALSE]

    # 归一化    归一化并不会直接在counts数值上修改，而是归一化系数会被自动存在y$samples$norm.factors!
    y = edgeR::calcNormFactors(y)

    # 指定一个dispersion值(设置BCV值)，这样得到的结果比较主观，不同的人就可以有不同的结果。通常如果是实验控制的好的人类数据，那么选择BCV=0.4，比较好的模式生物选择BCV=0.1
    bcv = 0.1
    gene_diff = edgeR::exactTest(y, dispersion=bcv^2)

    markers <- gene_diff$table
    markers$genes <- rownames(markers)
#     break
    #自定义阈值
    log2FC = 0.75
    padj = 0.05 

    markers$threshold="ns";
    markers[which(markers$logFC  > log2FC & markers$PValue <padj),]$threshold="up";
    markers[which(markers$logFC  < (-log2FC) & markers$PValue < padj),]$threshold="down";
    markers$threshold=factor(markers$threshold, levels=c('down','ns','up'))
    
    # Identify the top 99% of points based on -log10(PValue)
    bottom_5_percent <- quantile(markers$PValue, 0.001)
    # 将大于阈值的数字都改成阈值
    markers$PValue[markers$PValue < bottom_5_percent] <- bottom_5_percent
#     markers <- subset(markers, PValue >= bottom_5_percent)
#     print(paste0("第", i, "个markers处理完毕"))
    
    # 对 Top10 打标签
    top_logFC <- markers[order(markers$logFC), ][c(1:10, (nrow(markers)-9):nrow(markers)), ]
#     top_neg_logPValue <- markers[order(markers$PValue), ][c(1:5), ]
    
    plot <- ggplot(data=markers, aes(x=logFC, y=-log10(PValue), color=threshold)) +
              geom_point(alpha=0.8, size=0.8) +
              geom_vline(xintercept = c(-log2FC, log2FC), linetype=2, color="grey")+
              geom_hline(yintercept = -log10(padj), linetype=2, color="grey")+
              #labs(title= ifelse(""==title, "", paste("DEG:", title)))+
              xlab(bquote(Log[2]*FoldChange))+
              ylab(bquote(-Log[10]*italic(PValue)) )+
              theme_classic(base_size = 14) +
              scale_color_manual('',labels=c(paste0("down(",table(markers$threshold)[[1]],')'),'ns',
                                             paste0("up(",table(markers$threshold)[[3]],')' )),
                                 values=c("blue", "grey", "red"))+
              guides(color=guide_legend(override.aes = list(size=3, alpha=1))) +
              geom_text_repel(data=top_logFC, 
                aes(x = logFC, y = -log10(PValue), label = genes),
                box.padding = 0.1, 
                point.padding = 0.1,
                segment.color = "grey",
                segment.size = 0.2,
                segment.alpha = 0.5,
                direction = "y",
                force = 0.5,
                size = 1.5,
                color = "black")
#               geom_text(data=top_logFC, 
#                           aes(x = logFC, y = -log10(PValue), label = genes),
#                           vjust = 0, 
#                           hjust = 0,
#                           size=1.5,
#                           color = "black") + 
#               geom_text(data=top_neg_logPValue, 
#                           aes(x = logFC, y = -log10(PValue), label = genes),
#                           vjust = 0, 
#                           hjust = 0,
#                           size=1.5,
#                           color = "black")
    
    if( !dir.exists(paste0("./results/读取edgeR找出的不同合并域的差异基因再画火山图/", month_type)) ) {
        dir.create(paste0("./results/读取edgeR找出的不同合并域的差异基因再画火山图/", month_type), recursive = TRUE)
    }
    ggsave(paste0("./results/读取edgeR找出的不同合并域的差异基因再画火山图/", month_type, "/区域",  unique(datas@meta.data$STAligner)[i], ".pdf"), 
           plot, dpi = 600,
         height = 7, width = 7, limitsize = FALSE)
    ggsave(paste0("./results/读取edgeR找出的不同合并域的差异基因再画火山图/", month_type, "/区域",  unique(datas@meta.data$STAligner)[i], ".png"), 
           plot, dpi = 600,
         height = 7, width = 7, limitsize = FALSE)
    
    print(paste0("第", i, "个保存完毕"))
}

[1] "第1个保存完毕"
[1] "第2个保存完毕"


Warning message:
"ggrepel: 8 unlabeled data points (too many overlaps). Consider increasing max.overlaps"
Warning message:
"ggrepel: 8 unlabeled data points (too many overlaps). Consider increasing max.overlaps"


[1] "第3个保存完毕"
[1] "第4个保存完毕"


Warning message:
"ggrepel: 5 unlabeled data points (too many overlaps). Consider increasing max.overlaps"
Warning message:
"ggrepel: 5 unlabeled data points (too many overlaps). Consider increasing max.overlaps"


[1] "第5个保存完毕"


Warning message:
"ggrepel: 3 unlabeled data points (too many overlaps). Consider increasing max.overlaps"
Warning message:
"ggrepel: 5 unlabeled data points (too many overlaps). Consider increasing max.overlaps"


[1] "第6个保存完毕"
[1] "第7个保存完毕"


# 多组学差异基因绘图

In [ ]:
# 读取多组学数据
datas_all <- readRDS('D:/Users/zyzhang/Alzheimer/multiome/Batch1&Batch2/绘制差异基因与差异peaks的图/RNA&ATAC_cluster_batch1&batch2.RDS')
datas_all

In [ ]:
# gene_all <- data.frame()

# for (i in 1:length(datas)) {
#     gene <- datas[[i]]
#     gene$X <- rownames(gene)
#     rownames(gene) <- NULL
#     gene$month_type <- strsplit(names(datas)[i], '_')[[1]][1]
#     gene$cluster_name <- strsplit(names(datas)[i], '_')[[1]][2]
#     gene$sig <- strsplit(names(datas)[i], '_')[[1]][3]
#     gene_all <- rbind(gene_all, gene)         
# }

In [ ]:
datas_all@meta.data$cell_type <- plyr::mapvalues(datas_all@meta.data$seurat_clusters, from = c(1:19),
                                                 to = c("Oli", "Exc", "Exc", "Exc", "Mic", 
                                                        "Exc", "Inh", "Inh",  "Exc", "Inh", 
                                                        "Inh", "Inh", "Exc", "Exc", "Exc", 
                                                        "Exc", "OPC", "Exc", "Exc"))

In [ ]:
datas_all@meta.data$month_type <- substr(datas_all@meta.data$orig.ident, 2, 3)
datas_all@meta.data$if_disease <- substr(datas_all@meta.data$orig.ident, 4, 4)

In [ ]:
datas_all@meta.data$month_type <- plyr::mapvalues(datas_all@meta.data$month_type, 
                                                  from = c('01','03','06','15'), 
                                                  to = c('1M', '3M', '6M', '15M'))

In [ ]:
datas_all@meta.data$if_disease <- plyr::mapvalues(datas_all@meta.data$if_disease, 
                                                  from = c('A','W'), 
                                                  to = c('AD', 'WT'))

In [ ]:
datas_all <- subset(datas_all, month_type==month_type)

# 按照是否患病分组

In [ ]:
# 样本分组
case_clusters = list()
control_clusters = list()
datas_all_case = subset(datas_all, if_disease=='AD')
datas_all_control = subset(datas_all, if_disease=='WT')

for (i in 1:length(unique(datas_all@meta.data$cell_type))) {
    case_clusters[[i]] = subset(datas_all_case, cell_type==unique(datas_all@meta.data$cell_type)[i])
    control_clusters[[i]] = subset(datas_all_control, cell_type==unique(datas_all@meta.data$cell_type)[i])
}
names(case_clusters) <- unique(datas_all@meta.data$cell_type)
names(control_clusters) <- unique(datas_all@meta.data$cell_type)

In [ ]:
# 提取表达矩阵存放于列表之中
data_count_case = list()
data_count_control = list()

# 提取计数矩阵, 并转化为dataframe 以便后续处理操作
#  data @assays$RNA@counts存的是表达矩阵，  rownames (data @assays$RNA@meta.features)存的是相应的基因名
for (i in 1:length(unique(datas_all@meta.data$cell_type))) {
    data_count_case[[i]] = data.frame(case_clusters[[i]]@assays$RNA@counts, row.names = rownames(case_clusters[[i]]@assays$RNA@meta.features))
    data_count_control[[i]] = data.frame(control_clusters[[i]]@assays$RNA@counts, row.names = rownames(control_clusters[[i]]@assays$RNA@meta.features))
}

In [ ]:
# # 记录基因表达量和的列表
# gene_sum_case = list()
# gene_sum_control = list()

# # 按行求和： 也就是求每个基因的总表达量之和，然后重新创建一个dataframe，行名是基因，列名是样本， 也就是将矩阵压缩为一列
# for (i in 1:length(unique(datas_all@meta.data$cell_type))) {
#     gene_sum_case[[i]] = data.frame(rowSums(data_count_case[[i]]), row.names = rownames(data_count_case[[i]]))
#     colnames(gene_sum_case[[i]]) = paste0('case_cluster_',i)
#     gene_sum_control[[i]] = data.frame(rowSums(data_count_control[[i]]), row.names = rownames(data_count_control[[i]]))
#     colnames(gene_sum_control[[i]]) = paste0('control_cluster_',i)
# }
# 记录基因表达量和的列表
gene_sum_case = list()
gene_sum_control = list()

# 按行求和： 也就是求每个基因的总表达量之和，然后重新创建一个dataframe，行名是基因，列名是样本， 也就是将矩阵压缩为一列
for (i in 1:length(unique(datas_all@meta.data$cell_type))) {
    gene_sum_case[[i]] = data.frame(rowSums(data_count_case[[i]]), row.names = rownames(data_count_case[[i]]))
#     colnames(gene_sum_case[[i]]) = paste0('case_cluster_',i)
    colnames(gene_sum_case[[i]]) = paste0('case')
    gene_sum_control[[i]] = data.frame(rowSums(data_count_control[[i]]), row.names = rownames(data_count_control[[i]]))
#     colnames(gene_sum_control[[i]]) = paste0('control_cluster_',i)
    colnames(gene_sum_control[[i]]) = paste0('control')
}

In [ ]:
# 记录差异基因的列表
DE_analyze_up = list()
DE_analyze_down = list()

group = c('control', 'case')

for (i in 1:length(unique(datas_all@meta.data$cell_type))) {
    # 将要差异表达的两个矩阵merge到一起， 按照原先的行名（也就是基因名）
    count_after_merge = merge(gene_sum_case[[i]], gene_sum_control[[i]], by = 'row.names', all=TRUE)
    # 由于merge完以后会添加一列（row.names），因此需要重新取出第一列作为行名
    rownames(count_after_merge)=count_after_merge[,1]  
    # 删除第一列
    count_after_merge=count_after_merge[,-1]    
    # 将NA值改为0
    count_after_merge[is.na(count_after_merge)] = 0
    
    # 构建含有组别标记的DGEList
    y <- edgeR::DGEList(counts=count_after_merge, group=group)

    # 自动过滤低表达
    keep.exprs<- edgeR::filterByExpr(y, group=group)
    y <- y[keep.exprs,,keep.lib.sizes=FALSE]

    # 归一化    归一化并不会直接在counts数值上修改，而是归一化系数会被自动存在y$samples$norm.factors!
    y = edgeR::calcNormFactors(y)

    # 指定一个dispersion值(设置BCV值)，这样得到的结果比较主观，不同的人就可以有不同的结果。通常如果是实验控制的好的人类数据，那么选择BCV=0.4，比较好的模式生物选择BCV=0.1
    bcv = 0.1
    gene_diff = edgeR::exactTest(y, dispersion=bcv^2)

    markers <- gene_diff$table
    markers$genes <- rownames(markers)
    
#     break
    #自定义阈值
    log2FC = 0.75
    padj = 0.05 

    markers$threshold="ns";
    markers[which(markers$logFC  > log2FC & markers$PValue <padj),]$threshold="up";
    markers[which(markers$logFC  < (-log2FC) & markers$PValue < padj),]$threshold="down";
    markers$threshold=factor(markers$threshold, levels=c('down','ns','up'))
    
    
    # Identify the top 99% of points based on -log10(PValue)
    bottom_5_percent <- quantile(markers$PValue, 0.001)
    # 将大于阈值的数字都改成阈值
    markers$PValue[markers$PValue < bottom_5_percent] <- bottom_5_percent
#     markers <- subset(markers, PValue >= bottom_5_percent)
#     print(paste0("第", i, "个markers处理完毕"))
    
    # 对 Top10 打标签
    top_logFC <- markers[order(markers$logFC), ][c(1:10, (nrow(markers)-9):nrow(markers)), ]
#     top_neg_logPValue <- markers[order(markers$PValue), ][c(1:5), ]
    
    plot <- ggplot(data=markers, aes(x=logFC, y=-log10(PValue), color=threshold)) +
              geom_point(alpha=0.8, size=0.8) +
              geom_vline(xintercept = c(-log2FC, log2FC), linetype=2, color="grey")+
              geom_hline(yintercept = -log10(padj), linetype=2, color="grey")+
              #labs(title= ifelse(""==title, "", paste("DEG:", title)))+
              xlab(bquote(Log[2]*FoldChange))+
              ylab(bquote(-Log[10]*italic(PValue)) )+
              theme_classic(base_size = 14) +
              scale_color_manual('',labels=c(paste0("down(",table(markers$threshold)[[1]],')'),'ns',
                                             paste0("up(",table(markers$threshold)[[3]],')' )),
                                 values=c("blue", "grey", "red"))+
              guides(color=guide_legend(override.aes = list(size=3, alpha=1))) +
              geom_text_repel(data=top_logFC, 
                aes(x = logFC, y = -log10(PValue), label = genes),
                box.padding = 0.1, 
                point.padding = 0.1,
                segment.color = "grey",
                segment.size = 0.2,
                segment.alpha = 0.5,
                direction = "y",
                force = 0.5,
                size = 1.5,
                color = "black")
#               geom_text(data=top_logFC, 
#                           aes(x = logFC, y = -log10(PValue), label = genes),
#                           vjust = 0, 
#                           hjust = 0,
#                           size=1.5,
#                           color = "black") + 
#               geom_text(data=top_neg_logPValue, 
#                           aes(x = logFC, y = -log10(PValue), label = genes),
#                           vjust = 0, 
#                           hjust = 0,
#                           size=1.5,
#                           color = "black")
#     break
    
    if( !dir.exists(paste0("./results/读取edgeR找出的不同细胞类型的差异基因再画火山图/", month_type)) ) {
        dir.create(paste0("./results/读取edgeR找出的不同细胞类型的差异基因再画火山图/", month_type)))
    }
    ggsave(paste0("./results/读取edgeR找出的不同细胞类型的差异基因再画火山图/", month_type, "/细胞类型", 
                  unique(datas_all@meta.data$cell_type)[i], ".pdf"), 
           plot, dpi = 600,
         height = 7, width = 7, limitsize = FALSE)
    ggsave(paste0("./results/读取edgeR找出的不同细胞类型的差异基因再画火山图/", month_type, "/细胞类型", 
                  unique(datas_all@meta.data$cell_type)[i], ".png"), 
           plot, dpi = 600,
         height = 7, width = 7, limitsize = FALSE)
    
    print(paste0("第", i, "个保存完毕"))
}